In [1]:
import os
import glob
from tika import parser

pdf_dir = 'pdf/starwberry_file/EN'
file_names = glob.glob(pdf_dir + "/*.pdf")
    
texts = []
    
for file_name in file_names:
    text = parser.from_file(file_name)
    print(type(text["content"]))
    texts.append(text["content"])


<class 'str'>
<class 'str'>
<class 'str'>


In [2]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=40)

chunks = text_splitter.create_documents(texts)
print(len(chunks))
print(chunks)


Created a chunk of size 757, which is longer than the specified 200
Created a chunk of size 2338, which is longer than the specified 200
Created a chunk of size 643, which is longer than the specified 200
Created a chunk of size 1379, which is longer than the specified 200
Created a chunk of size 1282, which is longer than the specified 200
Created a chunk of size 375, which is longer than the specified 200
Created a chunk of size 1773, which is longer than the specified 200
Created a chunk of size 905, which is longer than the specified 200
Created a chunk of size 921, which is longer than the specified 200
Created a chunk of size 332, which is longer than the specified 200
Created a chunk of size 473, which is longer than the specified 200
Created a chunk of size 699, which is longer than the specified 200
Created a chunk of size 605, which is longer than the specified 200
Created a chunk of size 1393, which is longer than the specified 200
Created a chunk of size 755, which is longe

1839
[Document(page_content='The Colletotrichum gloeosporioides species complex\n\n\nCopyright CBS-KNAW Fungal Biodiversity Centre, P.O. Box 85167, 3508 AD Utrecht, The Netherlands.'), Document(page_content='You are free to share - to copy, distribute and transmit the work, under the following conditions:\nAttribution:  You must attribute the work in the manner specified by the author or licensor (but not in any way that suggests that they endorse you or your use of the work). \nNon-commercial: You may not use this work for commercial purposes. \nNo derivative works: You may not alter, transform, or build upon this work. \nFor any reuse or distribution, you must make clear to others the license terms of this work, which can be found at http://creativecommons.org/licenses/by-nc-nd/3.0/legalcode. Any of the above conditions can be waived if you get \npermission from the copyright holder. Nothing in this license impairs or restricts the author’s moral rights.'), Document(page_content='St\

In [3]:
import os

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embeddings_model = HuggingFaceEmbeddings(
    model_name = 'sentence-transformers/all-MiniLM-L6-v2',
    model_kwargs = {'device': 'cuda'},
    encode_kwargs = {'normalize_embeddings': False}
)

embeddings = embeddings_model.embed_query(chunks[0].page_content)
print(embeddings)

chunk_texts = [chunk.page_content for chunk in chunks]

embeddings = [embeddings_model.embed_query(chunk_text) for chunk_text in chunk_texts]
print(len(embeddings))

database_path = "vectorDB_test"

if not os.path.exists(database_path):
    os.makedirs(database_path)
os.chmod(database_path, 0o777)

chromadb = Chroma.from_documents(chunks, embeddings_model, persist_directory=database_path)
chromadb.persist()


/home/pcs4090/LLaMA/llama3/env310/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/pcs4090/LLaMA/llama3/env310/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/pcs4090/LLaMA/llama3/env310/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when pos

[0.0012281805975362659, -0.03645391762256622, -0.07646869868040085, -0.018600445240736008, 0.038699835538864136, -0.06452779471874237, -0.12434853613376617, 0.028512250632047653, 0.019129369407892227, -0.02494020387530327, 0.024591300636529922, -0.03413786739110947, -0.08946149796247482, -0.02422184869647026, -0.11154738068580627, -0.028157083317637444, 0.05769941955804825, 0.04781855270266533, -0.019106393679976463, -0.01970333233475685, 0.06387638300657272, 0.06048314645886421, -0.00835282914340496, -0.037547312676906586, 0.03619430586695671, -0.028318306431174278, -0.07719584554433823, -0.01536215003579855, 0.04187443107366562, -0.06050119549036026, -0.012356672435998917, 0.05777791142463684, 0.1060442253947258, -0.008837442845106125, 0.04410776495933533, 0.048779066652059555, 0.005131573881953955, -0.02007017470896244, 0.054716818034648895, 0.048388898372650146, -0.017486853525042534, -0.021391525864601135, 0.08337224274873734, -0.006949547678232193, -0.1390613168478012, 0.01612416

/home/pcs4090/LLaMA/llama3/env310/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [5]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embeddings_model = HuggingFaceEmbeddings(
    model_name = 'sentence-transformers/all-MiniLM-L6-v2',
    model_kwargs = {'device': 'cuda'},
    encode_kwargs = {'normalize_embeddings': False}
)

database_path = "vectorDB_test"

chromadb = Chroma(embedding_function=embeddings_model, persist_directory=database_path)

user_query = "What is Anthracnose caused by?"
print("user_query :")
print(user_query)
print()

results = chromadb.similarity_search_with_score(user_query, 50)
print("number of results searched : {}".format(len(results)))

unique_results = set()
final_results = []

for i in range(len(results)):
    content = results[i][0].page_content
    if content not in unique_results:
        unique_results.add(content)
        final_results.append((content, results[i][1]))
        
final_results.sort(key=lambda a: a[1])

print("number of unique results : {}".format(len(unique_results)))
print("=======================")

for i in range(min(50, len(final_results))):
    print("Result {}, score = {} :".format(i, final_results[i][1]))
    print(final_results[i][0])
    print("=======================")


user_query :
What is Anthracnose caused by?

number of results searched : 50
number of unique results : 50
Result 0, score = 0.5745149254798889 :
The disease cycles of anthracnose on different hosts have similar

components (Peres et al., 2005; De Silva et al., 2017); they are

generally polycyclic, with splash-borne asexual spores (conidia)
Result 1, score = 0.6799604892730713 :
(conidia) produced in acervuli are considered as the only inoculum

responsible for the development of anthracnose epidemics.

Exclusion of a possible (even though unlikely) ascosporic
Result 2, score = 0.7259851694107056 :
Wang, X. X., Wang, B., Liu, J. L., Chen, J., Cui, X. P., Jiang, H., et al. (2010). First
report of anthracnose caused by Colletotrichum gloeosporioides on ramie in China.
Plant Dis. 94, 1508–1508. doi: 10.1094/PDIS-08-10-0556
Result 3, score = 0.7263445854187012 :
Ocfemia, GO, Agati JA (1925). The cause of anthracnose of avocado, mango and 
upo in the Philippine Islands. Philippine Agricult